In [5]:
!pip install zarr
!pip install xarray

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [6]:
import os

zarr_path= "C:\\Users\\zhsim\\Desktop\\SeasFireCube_v3.zarr"
if os.path.exists(zarr_path):
    print("Zarr dataset exists!")
else:
    print("Zarr dataset not found.")

Zarr dataset exists!


In [7]:
import zarr
import xarray as xr

ds = xr.open_zarr(zarr_path, consolidated=True)

In [8]:
# Choose a valid variable name
vpd = ds["vpd"]

# View some details
print(vpd)

<xarray.DataArray 'vpd' (time: 966, latitude: 720, longitude: 1440)> Size: 4GB
[1001548800 values with dtype=float32]
Coordinates:
  * latitude   (latitude) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * longitude  (longitude) float64 12kB -179.9 -179.6 -179.4 ... 179.6 179.9
  * time       (time) datetime64[ns] 8kB 2001-01-01 2001-01-09 ... 2021-12-27
Attributes:
    aggregation:    Temporal | mean
    creator_notes:  This variable is calculated in-house,with Tetens equation...
    description:    Vapour-pressure deficit, or VPD, is the difference (defic...
    long_name:      Vapour Pressure Deficit
    units:          hPa


In [9]:
print(list(ds.data_vars))      # Data variables (not coordinates)

['area', 'biomes', 'cams_co2fire', 'cams_frpfire', 'drought_code_max', 'drought_code_mean', 'fcci_ba', 'fcci_ba_valid_mask', 'fcci_fraction_of_burnable_area', 'fcci_fraction_of_observed_area', 'fcci_number_of_patches', 'fwi_max', 'fwi_mean', 'gfed_ba', 'gfed_ba_valid_mask', 'gfed_region', 'gwis_ba', 'gwis_ba_valid_mask', 'lai', 'lccs_class_0', 'lccs_class_1', 'lccs_class_2', 'lccs_class_3', 'lccs_class_4', 'lccs_class_5', 'lccs_class_6', 'lccs_class_7', 'lccs_class_8', 'lsm', 'lst_day', 'mslp', 'ndvi', 'oci_ao', 'oci_censo', 'oci_ea', 'oci_epo', 'oci_gmsst', 'oci_nao', 'oci_nina34_anom', 'oci_pdo', 'oci_pna', 'oci_soi', 'oci_wp', 'pop_dens', 'rel_hum', 'skt', 'ssr', 'ssrd', 'sst', 'swvl1', 'swvl2', 'swvl3', 'swvl4', 't2m_max', 't2m_mean', 't2m_min', 'tp', 'vpd', 'ws10']


In [10]:
print(list(ds.coords))         # Coordinate variables (e.g., time, lat, lon)

['latitude', 'longitude', 'time']


In [11]:
print(ds['cams_co2fire'])   # Replace with any variable name

# Or get its attributes
print(ds['cams_co2fire'].attrs)

<xarray.DataArray 'cams_co2fire' (time: 966, latitude: 720, longitude: 1440)> Size: 4GB
[1001548800 values with dtype=float32]
Coordinates:
  * latitude   (latitude) float64 6kB 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * longitude  (longitude) float64 12kB -179.9 -179.6 -179.4 ... 179.6 179.9
  * time       (time) datetime64[ns] 8kB 2001-01-01 2001-01-09 ... 2021-12-27
Attributes:
    creator_notes:    Missing years filled with Nan. To convert to kg multipl...
    description:      The Global Fire Assimilation System (GFAS) assimilates ...
    downloaded from:  https://confluence.ecmwf.int/display/CKB/CAMS%3A+Global...
    long_name:        Wildfire flux of carbon dioxide
    missing_years:    2001 & 2002, see Creators Notes
    provider:         ECMWF CAMS Global Fire Assimilation System (GFAS)
    units:            kg m-2
{'creator_notes': 'Missing years filled with Nan. To convert to kg multiply by the variable square_meters.', 'description': 'The Global Fire Assimilation System

In [12]:
!pip install scikit-learn
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

In [ ]:
# Step 1: Select variables to include in ML model
vars_for_ml = ['swvl1', 'gfed_ba', 'vpd', 'ndvi', 'lai', 't2m_mean', 'rel_hum']

# Drop missing values and flatten spatial dimensions
df_list = []
for var in vars_for_ml:
    da = ds[var].load().stack(z=("latitude", "longitude")).to_dataframe().dropna()
    df_list.append(da)

# Merge all variables into a single DataFrame
data = pd.concat(df_list, axis=1).dropna().reset_index()

In [ ]:
# Step 2: Create lag features (1-month lag of burned area to simulate fire history)
data['gfed_ba_lag1'] = data.groupby('z')['gfed_ba'].shift(1)
data = data.dropna()

In [ ]:
# Step 3: Define features (X) and target (y)
features = ['gfed_ba_lag1', 'vpd', 'ndvi', 'lai', 't2m_mean', 'rel_hum']
target = 'swvl1'

X = data[features]
y = data[target]

In [ ]:
# Step 4: Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Step 5: Train Random Forest Regressor
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:

# Step 6: Evaluate model
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f"Model R²: {r2:.3f}")
print(f"RMSE: {rmse:.4f}")

In [ ]:
# Step 7: Feature Importance Plot
importances = model.feature_importances_
plt.figure(figsize=(8, 5))
plt.barh(features, importances)
plt.title("Feature Importance for Predicting Soil Moisture (swvl1)")
plt.xlabel("Importance")
plt.grid()
plt.tight_layout()
plt.show()

KeyboardInterrupt: 